In [1]:
import os,sys,signal, copy
import math
from contextlib import contextmanager

import pickle
import numpy as np                                       # fast vectors and matrices
import matplotlib.pyplot as plt                          # plotting
import matplotlib.ticker as ticker
from matplotlib.animation import ArtistAnimation

from scipy.fftpack import fft
from scipy.signal.windows import hann

sys.path.insert(0, '../')
import musicnetRaven as musicnet

from time import time

sys.path.insert(0,'../lib/')
import config
import diagnosticsP3
# import base_model

from sklearn.metrics import average_precision_score

os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'   # see issue #152
os.environ['CUDA_VISIBLE_DEVICES']='3'

import torch
from torch.nn.functional import conv1d, mse_loss
import torch.nn.functional as F
import torch.nn as nn
from torchcontrib.optim import SWA

from tqdm import tqdm
import mir_eval

from torchsummary import summary

%matplotlib inline

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
# Network Parameters
epochs = 20
train_size = 100000
test_size = 50000
lr = 5e-4
pitch_shift = 0
jitter = 0.
num_workers = 15

# lvl1 convolutions are shared between regions
m = 128
k = 512              # lvl1 nodes
n_fft = 4096              # lvl1 receptive field
window = 16384 # total number of audio samples?
stride = 512
batch_size = 100

In [3]:
regions = 1 + (window - n_fft)//stride

def worker_init(args):
    signal.signal(signal.SIGINT, signal.SIG_IGN) # ignore signals so parent can handle them
    np.random.seed(os.getpid() ^ int(time())) # approximately random seed for workers
kwargs = {'num_workers': num_workers, 'pin_memory': True, 'worker_init_fn': worker_init}

In [4]:
start = time()
root = '../../data/'
test_set = musicnet.MusicNet(root=root, train=False, download=True, refresh_cache=False, window=window, epoch_size=test_size, mmap=False)
print("Time used = ", time()-start)

Time used =  0.3503999710083008


In [5]:
test_loader = torch.utils.data.DataLoader(dataset=test_set,batch_size=batch_size,**kwargs)

In [6]:
def create_filtersv2(n_fft, freq_bins=None, low=50,high=6000, sr=44100, freq_scale='linear', mode="fft"):
    if freq_bins==None:
        freq_bins = n_fft//2+1
    
    s = torch.arange(0, n_fft, 1.)
    wsin = torch.empty((freq_bins,1,n_fft))
    wcos = torch.empty((freq_bins,1,n_fft))
    start_freq = low
    end_freq = high
    

    # num_cycles = start_freq*d/44000.
    # scaling_ind = np.log(end_freq/start_freq)/k
    
    if mode=="fft":
        window_mask = 1
    elif mode=="stft":
        window_mask = 0.5-0.5*torch.cos(2*math.pi*s/(n_fft)) # same as hann(n_fft, sym=False)
    else:
        raise Exception("Unknown mode, please chooes either \"stft\" or \"fft\"")
        
    if freq_scale == 'linear':
        start_bin = start_freq*n_fft/sr
        scaling_ind = (end_freq/start_freq)/freq_bins
        for k in range(freq_bins): # Only half of the bins contain useful info
            wsin[k,0,:] = window_mask*torch.sin(2*math.pi*(k*scaling_ind*start_bin)*s/n_fft)
            wcos[k,0,:] = window_mask*torch.cos(2*math.pi*(k*scaling_ind*start_bin)*s/n_fft)
    elif freq_scale == 'log':
        start_bin = start_freq*n_fft/sr
        scaling_ind = np.log(end_freq/start_freq)/freq_bins
        for k in range(freq_bins): # Only half of the bins contain useful info
            wsin[k,0,:] = window_mask*torch.sin(2*math.pi*(np.exp(k*scaling_ind)*start_bin)*s/n_fft)
            wcos[k,0,:] = window_mask*torch.cos(2*math.pi*(np.exp(k*scaling_ind)*start_bin)*s/n_fft)    
    else:
        print("Please select the correct frequency scale, 'linear' or 'log'")
    
    return wsin,wcos


In [7]:
class Model(nn.Module):
    def __init__(self, avg=.9998):
        super(Model, self).__init__()
        # Create filter windows
        wsin, wcos = create_filtersv2(n_fft,k, low=50, high=6000,
                                      mode="stft", freq_scale='log')
        with torch.cuda.device(0):
            self.wsin = torch.Tensor(wsin).cuda()
            self.wcos = torch.Tensor(wcos).cuda()
            
        # Creating Layers
#         self.linear = torch.nn.Linear(regions*k, k,bias=False)
#         self.linear_output = torch.nn.Linear(k,m, bias=False)
#         wscale = 10e-5
#         torch.nn.init.normal_(self.linear.weight, std=1e-4) # initialize
#         torch.nn.init.normal_(self.linear_output.weight, std=1e-4)
#         torch.nn.init.zeros_(self.linear.weight)
#         torch.nn.init.zeros_(self.linear_output.weight)
        k_out = 128
        k2_out = 256
        stride1 = (2,1)
        self.CNN_freq = nn.Conv2d(1,k_out,
                                kernel_size=(128,1),stride=(2,1))
        self.CNN_time = nn.Conv2d(k_out,k2_out,
                                kernel_size=(1,25),stride=(1,1))
        self.Linear = nn.Linear(k2_out*1*193, m)
        self.activation = nn.ReLU()
        
        torch.nn.init.normal_(self.CNN_freq.weight, std=1e-4)
        torch.nn.init.normal_(self.CNN_time.weight, std=1e-4)
        torch.nn.init.normal_(self.Linear.weight, std=1e-4)
        
        
        self.avg = avg
        #Create a container for weight average
        self.averages = copy.deepcopy(list(parm.cuda().data for parm in self.parameters())) 
    def _get_conv_output(self, shape):
        bs = 1
        x = torch.tensor(torch.rand(bs, *shape))
        output_feat = self._forward_features(x)
        n_size = output_feat.data.view(bs, -1).size(1)
        return n_size
        
    def forward(self,x):
        zx = conv1d(x[:,None,:], self.wsin, stride=stride).pow(2) \
           + conv1d(x[:,None,:], self.wcos, stride=stride).pow(2)
        zx = torch.log(zx + 10e-8) # Log Magnitude Spectrogram
        z2 = self.CNN_freq(zx.unsqueeze(1)) # Make channel as 1 (N,C,H,W)
        z2 = F.relu(z2)
        z3 = self.CNN_time(z2)
        z3 = F.relu(z3)
        y = self.Linear(self.activation(torch.flatten(z3,1)))
        return y
    
    def average_iterates(self):
        for parm, pavg in zip(self.parameters(), self.averages):
            pavg.mul_(self.avg).add_(1.-self.avg, parm.data) # 0.9W_avg + 0.1W_this_ite
 

    
@contextmanager
def averages(model):
    orig_parms = copy.deepcopy(list(parm.data for parm in model.parameters()))
    for parm, pavg in zip(model.parameters(), model.averages):
        parm.data.copy_(pavg)
    yield
    for parm, orig in zip(model.parameters(), orig_parms):
        parm.data.copy_(orig)

In [8]:
class MultiLayerModel(torch.nn.Module):
    def __init__(self, avg=.9998):
        super(MultiLayerModel, self).__init__()
        # Create filter windows
        wsin, wcos = create_filtersv2(n_fft,k, low=50, high=6000,
                                      mode="stft", freq_scale='log')
        with torch.cuda.device(0):
            self.wsin = torch.Tensor(wsin).cuda()
            self.wcos = torch.Tensor(wcos).cuda()
            
        # Creating Layers
        self.linear = torch.nn.Linear(regions*k, k,bias=False)
        self.linear_output = torch.nn.Linear(k,m, bias=False)
        wscale = 10e-5
        torch.nn.init.normal_(self.linear.weight, std=1e-4) # initialize
        torch.nn.init.normal_(self.linear_output.weight, std=1e-4)
#         torch.nn.init.zeros_(self.linear.weight)
#         torch.nn.init.zeros_(self.linear_output.weight)
        self.activation = torch.nn.ReLU()
        
        self.avg = avg
        #Create a container for weight average
        self.averages = copy.deepcopy(list(parm.cuda().data for parm in self.parameters())) 

        
    def forward(self,x):
        zx = conv1d(x[:,None,:], self.wsin, stride=stride).pow(2) \
           + conv1d(x[:,None,:], self.wcos, stride=stride).pow(2)
        zx = self.activation(self.linear(torch.log(zx + 10e-8).view(x.data.size()[0],regions*k)))
        output = self.linear_output(zx)
        return output
    
    def average_iterates(self):
        for parm, pavg in zip(self.parameters(), self.averages):
            pavg.mul_(self.avg).add_(1.-self.avg, parm.data) # 0.9W_avg + 0.1W_this_ite

In [9]:
class CNNModel(nn.Module):
    def __init__(self, avg=.9998):
        super(CNNModel, self).__init__()
        # Create filter windows
        wsin, wcos = create_filtersv2(n_fft,k, low=50, high=6000,
                                      mode="stft", freq_scale='log')
        with torch.cuda.device(0):
            self.wsin = torch.Tensor(wsin).cuda()
            self.wcos = torch.Tensor(wcos).cuda()
            
        # Creating Layers
#         self.linear = torch.nn.Linear(regions*k, k,bias=False)
#         self.linear_output = torch.nn.Linear(k,m, bias=False)
#         wscale = 10e-5
#         torch.nn.init.normal_(self.linear.weight, std=1e-4) # initialize
#         torch.nn.init.normal_(self.linear_output.weight, std=1e-4)
#         torch.nn.init.zeros_(self.linear.weight)
#         torch.nn.init.zeros_(self.linear_output.weight)
        k_out = 128
        k2_out = 256
        stride1 = (2,1)
        self.CNN_freq = nn.Conv2d(1,k_out,
                                kernel_size=(128,1),stride=(2,1))
        self.CNN_time = nn.Conv2d(k_out,k2_out,
                                kernel_size=(1,25),stride=(1,1))
        self.Linear = nn.Linear(k2_out*1*193, m)
        self.activation = nn.ReLU()
        
        torch.nn.init.normal_(self.CNN_freq.weight, std=1e-4)
        torch.nn.init.normal_(self.CNN_time.weight, std=1e-4)
        torch.nn.init.normal_(self.Linear.weight, std=1e-4)
        
        
        self.avg = avg
        #Create a container for weight average
        self.averages = copy.deepcopy(list(parm.cuda().data for parm in self.parameters())) 
    def _get_conv_output(self, shape):
        bs = 1
        x = torch.tensor(torch.rand(bs, *shape))
        output_feat = self._forward_features(x)
        n_size = output_feat.data.view(bs, -1).size(1)
        return n_size
        
    def forward(self,x):
        zx = conv1d(x[:,None,:], self.wsin, stride=stride).pow(2) \
           + conv1d(x[:,None,:], self.wcos, stride=stride).pow(2)
        zx = torch.log(zx + 10e-8) # Log Magnitude Spectrogram
        z2 = self.CNN_freq(zx.unsqueeze(1)) # Make channel as 1 (N,C,H,W)
        z2 = F.relu(z2)
        z3 = self.CNN_time(z2)
        z3 = F.relu(z3)
        y = self.Linear(self.activation(torch.flatten(z3,1)))
        return y
    
    def average_iterates(self):
        for parm, pavg in zip(self.parameters(), self.averages):
            pavg.mul_(self.avg).add_(1.-self.avg, parm.data) # 0.9W_avg + 0.1W_this_ite

# Averaged Weights

In [10]:
model = CNNModel()
model.cuda()

result_dict = {'loss_history_train': [],
               'avgp_history_train': [],
               'loss_history_test': [],
               'avgp_history_test': [],
               'parameters': {}}

result_dict['parameters']['train_size'] = train_size
result_dict['parameters']['test_size'] = test_size
result_dict['parameters']['lr'] = lr
result_dict['parameters']['pitch_shift'] = pitch_shift
result_dict['parameters']['jitter'] = jitter

In [11]:
model.load_state_dict(torch.load('./Transistion Invariant/weights/2 Layers CNN-Tried Another Optim'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

# Mirex stats

In [12]:
from pypianoroll import Multitrack, Track, load, parse

def get_mir_accuracy(Yhat, Y_true, threshold=0.4):
    Yhatlist = []
    Ylist = []
    Yhatpred = Yhat>threshold
    for i in range(len(Yhatpred)):
        print(f"{i}/{len(Yhatpred)} batches", end = '\r')
        fhat = []
        f = []
        for note in range(m):
            if Yhatpred[i][note] == 1:
                fhat.append(440.*2**(((note)-69.)/12.))

            if Y_true[i][note] == 1:
                f.append(440.*2**(((note)-69.)/12.))

        Yhatlist.append(np.array(fhat))
        Ylist.append(np.array(f))
    avp = average_precision_score(Y_true.flatten(),Yhat.flatten())
    P,R,Acc,Esub,Emiss,Efa,Etot,cP,cR,cAcc,cEsub,cEmiss,cEfa,cEtot = \
    mir_eval.multipitch.metrics(np.arange(len(Ylist))/100.,Ylist,np.arange(len(Yhatlist))/100.,Yhatlist)
    print('{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}'.format(100*avp,100*P,100*R,Acc,Etot,Esub,Emiss,Efa))
    return avp,P,R,Acc,Etot
def get_piano_roll(rec_id, window=16384, stride=1000, offset=44100, count=7500):
    sf=4
    if stride == -1:
        stride = (test_set.records[rec_id][1] - offset - int(sf*window))/(count-1)
        stride = int(stride)
    else:
        count = (test_set.records[rec_id][1] - offset - int(sf*window))/stride + 1
        count = int(count)

    X = np.zeros([count, window])
    Y = np.zeros([count, m])    

    for i in range(count):
        X[i,:], Y[i] =  test_set.access(rec_id, offset+i*stride)

    batch_size = 500
    Y_pred = np.zeros([count,m])
    output = []
    for i in range(len(X)//500):
        X_batch = torch.tensor(X[batch_size*i:batch_size*(i+1)]).float().cuda()   
        Y_pred[i*batch_size:(i+1)*batch_size] = model(X_batch).data.cpu().numpy()
    
    return Y_pred, Y

def export_midi(Y_pred, path):
    # Create a piano-roll matrix, where the first and second axes represent time
    # and pitch, respectively, and assign a C major chord to the piano-roll
    # Create a `pypianoroll.Track` instance
    track = Track(pianoroll=Y_pred*127, program=0, is_drum=False,
                  name='my awesome piano')   
    multitrack = Multitrack(tracks=[track], tempo=120.0, beat_resolution=24)
    multitrack.write(path)    

In [13]:
with test_set:
    print('AvgP\tP\tR\tAcc\tETot\tESub\tEmiss\tEfa')
    model.eval()
    Accavg = 0
    Etotavg = 0
    for songid in test_set.rec_ids:
        Y_pred, Y_true = get_piano_roll(songid, stride=-1)
    #     Yhatpred = Y_pred > 0.4
        _,_,_,Acc,Etot = get_mir_accuracy(Y_pred, Y_true)
        Accavg += Acc
        Etotavg += Etot

AvgP	P	R	Acc	ETot	ESub	Emiss	Efa


RuntimeError: CUDA out of memory. Tried to allocate 1.15 GiB (GPU 0; 31.74 GiB total capacity; 1.28 GiB already allocated; 114.12 MiB free; 24.08 MiB cached)

In [ ]:
print('Average Accuracy: \t{:2.2f}\nAverage Error: \t\t{:2.2f}'.format(Accavg*100/len(test_set.rec_ids), Etotavg*100/len(test_set.rec_ids)))

In [38]:
songids = [1759, 2106, 2382, 2556]

In [39]:
for songid in songids:
    model.eval()
    Y_pred, Y_true = get_piano_roll(songid, stride=-1)
    _,_,_,Acc,Etot = get_mir_accuracy(Y_pred, Y_true)
    
    Yhatpred = Y_pred > 0.4
    export_midi(Yhatpred, './{}_{}_Y_pred.mid'.format('CNN_MSE',str(songid)))

712/7500 batches

1392/7500 batches

2058/7500 batches

3057/7500 batches

4056/7500 batches

74.34	69.86	74.10	0.56	0.46	0.12	0.14	0.20
70.29	72.59	63.38	0.51	0.47	0.13	0.23	0.11
48.63	55.47	47.18	0.34	0.74	0.17	0.36	0.21
77.45	74.57	74.33	0.59	0.39	0.12	0.14	0.14
